## 例一：平面拟合

In [1]:
import tensorflow as tf
import numpy as np

In [4]:
#用numpy 生成假数据
x_data = np.float32(np.random.rand(2, 100))
y_data = np.dot([0.1, 0.2], x_data) + 0.3

In [6]:
# 构造线下模型
b = tf.Variable(tf.zeros([1]))
W = tf.Variable(tf.random_uniform([1, 2], -1.0, 1.0))
y = tf.matmul(W, x_data) + b

In [7]:
# 3.构建模型框架
loss = tf.reduce_mean(tf.square(y - y_data))
# 优化方法，选择梯度下降
optimizer = tf.train.GradientDescentOptimizer(0.5)
# 迭代的目标：最小化损失函数
train = optimizer.minimize(loss)

In [8]:
# 利用数据，求解模型参数
# 1.初始化变量：tf 的必备步骤，主要声明了变量，就必须初始化才能用
init = tf.global_variables_initializer()
# 设置tensorflow对GPU的使用按需分配
config = tf.ConfigProto()
config.gpu_options.allow_growth = False
# 2.启动图 (graph)
sess = tf.Session(config=config)
sess.run(init)

In [11]:
# 3.迭代，反复执行上面的最小化损失函数这一操作（train op）,拟合平面
for step in range(0, 201):
    sess.run(train)
    if step % 20 == 0:
        print(step, sess.run(W), sess.run(b))

0 [[ 0.10000011  0.20000008]] [ 0.29999989]
20 [[ 0.10000011  0.20000008]] [ 0.29999989]
40 [[ 0.10000011  0.20000008]] [ 0.29999989]
60 [[ 0.10000011  0.20000008]] [ 0.29999989]
80 [[ 0.10000011  0.20000008]] [ 0.29999989]
100 [[ 0.10000011  0.20000008]] [ 0.29999989]
120 [[ 0.10000011  0.20000008]] [ 0.29999989]
140 [[ 0.10000011  0.20000008]] [ 0.29999989]
160 [[ 0.10000011  0.20000008]] [ 0.29999989]
180 [[ 0.10000011  0.20000008]] [ 0.29999989]
200 [[ 0.10000011  0.20000008]] [ 0.29999989]


## 例二：两个数求和

In [13]:
input1 = tf.constant(2.0)
input2 = tf.constant(3.0)
input3 = tf.constant(5.0)

intermd = tf.add(input1, input2)
mul = tf.multiply(input2, input3)

In [14]:
with tf.Session() as sess:
    result = sess.run([mul, intermd])  # 一次执行多个op
    print(result)
    print(type(result))
    print(type(result[0]))   

[15.0, 5.0]
<class 'list'>
<class 'numpy.float32'>


## 知识点学习

### 1.1 用 tensorflow 实现计数器，主要是设计了在循环中调用加法实现计数

In [16]:
# 创建变量，初始化为0
state = tf.Variable(0, name="counter")
# 创建一个op , 其作用是使 state 增加 1
one = tf.constant(1) # 直接用1
new_value = tf.add(state, 1)
update = tf.assign(state, new_value)

In [19]:
# 启动图之后，运行 update op
with tf.Session() as sess:
    # 创建好图之后，变量必须经过‘初始化’ 
    sess.run(tf.global_variables_initializer())
    # 查看state的初始化值
    print(sess.run(state))
    for _ in range(3):
        sess.run(update)
        print(sess.run(state))

0
1
2
3


### 1.2 用 tf 来实现对一组数求和，再计算平均

In [23]:
# 知识点：tf.Variable和tf.placeholder的区别
# discuss link:https://stackoverflow.com/questions/36693740/whats-the-difference-between-tf-placeholder-and-tf-variable
# tf.Variable：主要在于一些可训练变量（trainable variables），比如模型的权重（weights，W）或者偏执值（bias）
# (1). 声明时，必须提供初始值；
# (2). 名称的真实含义，在于变量，也即在真实训练时，其值是会改变的，自然事先需要指定初始值；
# tf.placeholder：用于得到传递进来的真实的训练样本：
# (1). 不必指定初始值，可在运行时，通过 Session.run 的函数的 feed_dict 参数指定；
# (2). 这也是其命名的原因所在，仅仅作为一种占位符；

In [20]:
h_sum = tf.Variable(0.0, dtype=tf.float32)
# h_vec = tf.random_normal(shape=([10]))
h_vec = tf.constant([1.0, 2.0, 3.0, 4.0])
# 把 h_vec 的每个元素加到 h_sum 中，然后再除以 10 来计算平均值
# 待添加的数
h_add = tf.placeholder(tf.float32)
# 添加之后的值
h_new = tf.add(h_sum, h_add)
# 更新 h_new 的 op
update = tf.assign(h_sum, h_new)

In [22]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # 查看原始值
    print('s_sum =', sess.run(h_sum))
    print("vec = ", sess.run(h_vec))

    # 循环添加
    for _ in range(4):
        sess.run(update, feed_dict={h_add: sess.run(h_vec[_])})
        print('h_sum =', sess.run(h_sum))
    #print 'the mean is ', sess.run(sess.run(h_sum) / 4)  # 这样写 4  是错误的， 必须转为 tf 变量或者常量
    print('the mean is ', sess.run(sess.run(h_sum) / tf.constant(4.0)))

s_sum = 0.0
vec =  [ 1.  2.  3.  4.]
h_sum = 1.0
h_sum = 3.0
h_sum = 6.0
h_sum = 10.0
the mean is  2.5


## 2. InteractiveSession() 的用法

In [29]:
# InteractiveSession() 主要是避免 Session（会话）被一个变量持有
a = tf.constant(1.0)
b = tf.constant(2.0)
c = a + b

# 下面的两种情况是等价的
with tf.Session():
    print(c.eval())

sess = tf.InteractiveSession()
print(c.eval())
sess.close()

3.0
3.0


In [33]:
a = tf.constant(1.0)
b = tf.constant(2.0)
c = tf.Variable(3.0)
d = a + b

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
###################
# 这样写是错误的
# print a.run()  
# print d.run()
####################

# 这样才是正确的
# print(a.eval())   
# print(d.eval())

# run() 方法主要用来
x = tf.Variable(1.2)
# print x.eval()  # 还没初始化，不能用
x.initializer.run()  # x.initializer 就是一个初始化的 op， op才调用run() 方法
print(x.eval())

sess.close()

1.2


In [34]:
### 2.1 使用 tf.InteractiveSession() 来完成上面 1.2中求和、平均的操作?

In [35]:
h_sum = tf.Variable(0.0, dtype=tf.float32)
# h_vec = tf.random_normal(shape=([10]))
h_vec = tf.constant([1.0, 2.0, 3.0, 4.0])
# 把 h_vec 的每个元素加到 h_sum 中，然后再除以 10 来计算平均值
# 待添加的数
h_add = tf.placeholder(tf.float32)
# 添加之后的值
h_new = tf.add(h_sum, h_add)
# 更新 h_new 的 op
update = tf.assign(h_sum, h_new)

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
print('s_sum =', h_sum.eval())
print("vec = ", h_vec.eval())
print("vec = ", h_vec[0].eval())


for _ in range(4):
    update.eval(feed_dict={h_add: h_vec[_].eval()})
    print('h_sum =', h_sum.eval())
sess.close()

s_sum = 0.0
vec =  [ 1.  2.  3.  4.]
vec =  1.0
h_sum = 1.0
h_sum = 3.0
h_sum = 6.0
h_sum = 10.0


## 3.使用feed来对变量赋值

In [36]:
# 这些需要用到feed来赋值的操作可以通过tf.placeholder()说明，以创建占位符。 

# 下面的例子中可以看出 session.run([output], …) 和 session.run(output, …) 的区别。前者输出了 output 的类型等详细信息，后者只输出简单结果。

### 3.1 session.run([output], …)

In [38]:
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = tf.multiply(input1, input2)

with tf.Session() as sess:
    print(sess.run([output], feed_dict={input1:[7.0], input2:[2.0]}))

[array([ 14.], dtype=float32)]


In [41]:
with tf.Session() as sess:
    print(sess.run([output], feed_dict={input1:[7.0, 3.0], input2:[2.0, 1.0]}))

[array([ 14.,   3.], dtype=float32)]


### 3.2 session.run(output, …)

In [39]:
with tf.Session() as sess:
    result = sess.run(output, feed_dict={input1:[7.0], input2:[2.0]})
    print(type(result))
    print(result)

<class 'numpy.ndarray'>
[ 14.]


In [42]:
with tf.Session() as sess:
    print(sess.run(output, feed_dict={input1:[7.0, 3.0], input2:[2.0, 1.0]}))

[ 14.   3.]
